In [26]:
from extracting_data import extract
from utilis import change_date, change_genre
import pandas as pd

def change_string(x):
    x = x.replace('{', '')
    x = x.replace('}', '')
    x = x.split(',')
    return x


def tranform_column(column):
    d = column.apply(change_string)
    list_of_dictionaries = []
    for row in d:
        data_dict = {}
        for item in row:
            item = item.replace("'", "").strip()
            key, value = item.split(':', 1)
            data_dict[key.strip()] = value.strip()
        list_of_dictionaries.append(data_dict)

    return list_of_dictionaries


def create_new_column(column, df):
    l = tranform_column(column)

    danceability = []
    energy = []
    loudness = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    liveness = []
    valence = []
    tempo = []

    for dic in l:
        danceability.append(dic['danceability'])
        energy.append(dic['energy'])
        loudness.append(dic['loudness'])
        speechiness.append(dic['speechiness'])
        acousticness.append(dic['acousticness'])
        instrumentalness.append(dic['instrumentalness'])
        liveness.append(dic['liveness'])
        valence.append(dic['valence'])
        tempo.append(dic['tempo'])

    df['danceability'] = danceability
    df['energy'] = energy
    df['loudness'] = loudness
    df['speechiness'] = speechiness
    df['acousticness'] = acousticness
    df['instrumentalness'] = instrumentalness
    df['liveness'] = liveness
    df['valence'] = valence
    df['tempo'] = tempo

    df.drop(columns=['other'], inplace=True)

    return df

def get_playlist_data(link, file_name): # tu będziemy otrzymać encodowane dane do rekomendacji i new_data czyli dane bez encodowania, które będą przydatne podczas analizy playlisty 
    extract(link, f'{file_name}.csv')
    new_data = pd.read_csv(f'{file_name}.csv')
    new_data.dropna(inplace=True, subset=['other'])
    create_new_column(new_data['other'], new_data)
    new_data.drop([new_data.columns[0]], axis=1, inplace=True)
    new_data['year'] = new_data['release date'].apply(change_date)
    new_data.drop(columns='release date', inplace=True)
    new_data['track genre'].fillna('Inne', inplace=True)
    new_data['new genre'] = new_data['track genre'].apply(change_genre)
    new_data.drop(columns='track genre', inplace=True)
    encoded_data = pd.get_dummies(new_data, columns=['new genre'], drop_first=True, dtype=int)
    return encoded_data, new_data

In [39]:
rock_playlist = 'https://open.spotify.com/playlist/041EEjr8FMkWlzbuKnSXYD?si=161e174cef984d55'
playlist_encoded, playlist = get_playlist_data(rock_playlist, 'rap_test')

C:\Users\HP\AppData\Local\Temp\ipykernel_1672\581378736.py:72: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  new_data['track genre'].fillna('Inne', inplace=True)


In [30]:
playlist

,track,artist,album,artist pop,track pop,duration,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,year,new genre
0,In the End,Linkin Park,Hybrid Theory (Bonus Edition),83,88,216880,0.556,0.864,-5.87,0.0584,0.00958,0,0.209,0.4,105.143,2000,metal
1,Have You Ever Seen The Rain,Creedence Clearwater Revival,Pendulum (Expanded Edition),79,87,160133,0.741,0.697,-7.028,0.0277,0.0664,2.28e-05,0.133,0.774,116.109,1970,rock
2,Every Breath You Take,The Police,Synchronicity (Remastered 2003),74,88,253920,0.82,0.452,-9.796,0.0348,0.543,0.00294,0.0714,0.74,117.401,1983,rock
3,Chop Suey!,System Of A Down,Toxicity,78,85,210240,0.417,0.934,-3.908,0.119,0.000278,0.0015,0.132,0.287,127.066,2001,metal
4,"Paint It, Black",The Rolling Stones,Aftermath,77,84,202266,0.464,0.795,-9.267,0.0926,0.0493,0.00244,0.399,0.612,158.691,1966,rock
5,Highway to Hell,AC/DC,Highway to Hell,80,86,208400,0.574,0.913,-4.793,0.133,0.061,0.00158,0.156,0.423,115.728,1979,rock
6,Dream On,Aerosmith,Aerosmith,73,82,267596,0.392,0.476,-7.583,0.0249,0.397,0.000163,0.309,0.234,78.031,1973,rock
7,Killing In The Name,Rage Against The Machine,Rage Against The Machine - XX (20th Anniversar...,68,79,313573,0.466,0.833,-4.215,0.304,0.0266,0,0.0327,0.661,88.785,1992,metal
8,Californication,Red Hot Chili Peppers,Californication (Deluxe Edition),80,83,329733,0.592,0.767,-2.788,0.027,0.0021,0.00165,0.127,0.328,96.483,1999,rock
9,Toxicity,System Of A Down,Toxicity,78,82,218933,0.413,0.873,-4.151,0.0536,0.000264,0.00249,0.204,0.48,116.68,2001,metal


In [40]:
playlist_num = playlist_encoded.drop(columns=['artist', 'album', 'track'])

In [41]:
from sklearn.preprocessing import MinMaxScaler

def scale_data(data, column_names):
    df = pd.DataFrame(data, columns=column_names)
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)
    scaled_df = pd.DataFrame(scaled_data, columns=column_names)
    return scaled_df

In [42]:
scaled_playlist = scale_data(data=playlist_num, column_names=playlist_num.columns)

In [43]:
spotify_data = pd.read_csv('spotify_data_encoded2.csv')
spotify_data.drop([spotify_data.columns[0]], axis=1, inplace=True)
spotify_data_num = spotify_data.drop(columns=['track', 'artist', 'album'])
spotify_data_scaled = scale_data(spotify_data_num, spotify_data_num.columns)

# Liczenie średnich wartości

In [44]:
column_averages = scaled_playlist.mean()
averages_cosine_sim = pd.DataFrame([column_averages], index=['Average'])
averages_cosine_sim

,artist pop,track pop,duration,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,year,new genre_other,new genre_pop,new genre_rap
Average,0.692043,0.662159,0.423539,0.652978,0.395312,0.509461,0.345078,0.19043,0.01732,0.209487,0.414427,0.447205,0.702143,0.32,0.02,0.29


# Synchronizacja kolumn

In [45]:
def synchronize_columns(df1, df2):
    missing_in_df2 = set(df1.columns) - set(df2.columns)
    missing_in_df1 = set(df2.columns) - set(df1.columns)
    
    for col in missing_in_df2:
        df2[col] = 0
        
    for col in missing_in_df1:
        df1[col] = 0
        
    df1 = df1[sorted(df1.columns)]
    df2 = df2[sorted(df2.columns)]
    
    return df1, df2

In [46]:
synchronize_columns(spotify_data_scaled, averages_cosine_sim)
averages_cosine_sim = averages_cosine_sim.sort_index(axis=1)
spotify_data_scaled = spotify_data_scaled.sort_index(axis=1)

In [48]:
averages_cosine_sim

,acousticness,artist pop,danceability,duration,energy,instrumentalness,liveness,loudness,new genre_classical,new genre_dance,...,new genre_other,new genre_pop,new genre_rap,new genre_rock,new genre_soul,speechiness,tempo,track pop,valence,year
Average,0.19043,0.692043,0.652978,0.423539,0.395312,0.01732,0.209487,0.509461,0,0,...,0.32,0.02,0.29,0,0,0.345078,0.447205,0.662159,0.414427,0.702143


# Rekomendacja nowych 20 piosenek 

In [47]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_scores = cosine_similarity(spotify_data_scaled, averages_cosine_sim)
 
spotify_data['similarity_score'] = similarity_scores
 
new_frame = pd.merge(spotify_data, playlist, how='left', on='track')
top_similarities_filtered = new_frame[new_frame.isna().any(axis=1)]
top_similarities_filtered = top_similarities_filtered.sort_values(by='similarity_score', ascending=False)
top_similarities_filtered = top_similarities_filtered.drop_duplicates(subset=['track', 'artist_x'])
top_similarities_filtered[['track', 'artist_x', 'similarity_score']].head(20)


,track,artist_x,similarity_score
1086,Starboy,The Weeknd,0.940790
1131,SICKO MODE,Travis Scott,0.935822
1455,I Feel It Coming,The Weeknd,0.930468
2015,Wasted (feat. Lil Uzi Vert),Juice WRLD,0.929872
1375,Die For You,The Weeknd,0.928569
617,Rhyme Dust,MK,0.927913
1998,Still Think About You,A Boogie Wit da Hoodie,0.926612
2000,Jocelyn Flores,XXXTENTACION,0.926083
546,Pedro,Jaxomy,0.925779
1960,WAKE UP,Travis Scott,0.925573


# 2 wersja